# TripAdvisor Data Acquisition and Management
### (1) develop code that downloads the page to the local file system
#### Following code download the page of each available hotel in Los Angeles in TripAdvisor

In [1]:
import requests, re, math
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import json
import os
import mysql.connector
from mysql.connector import errorcode
import json
from sqlalchemy import create_engine
import pymysql
%load_ext autotime

In [2]:
#Write a function to extract parse the html of the url.
def get_html(url):
    headers = {
   # pretend I am a browser
   'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.87 Safari/537.36',
   }
    session = requests.Session()
    data = session.get(url, headers=headers)
    html = data.text
    return html

# Write a function that get the links of hotels from the search result page
def get_link(url):
    links = list()
    soup = BeautifulSoup(get_html(url))
    hotels = soup.find(id = 'taplc_hsx_hotel_list_lite_dusty_hotels_combined_sponsored_0').find_all('div', {'class' : 'prw_rup prw_meta_hsx_responsive_listing ui_section listItem'})
    for hotel in hotels:
        link = re.findall(r'<a.+?class="respListingPhoto".+?href="(/Hotel.+?)">',str(hotel))
        link = 'https://www.tripadvisor.com/'+link[0]
        links.append(link)
    return links

time: 1e+03 µs


In [3]:
# Notice that each page in TripAdvisor shows 30 hotels. In total there are 30 pages for hotels in Los Angeles, so we have to access
# 30 urls.

# Generate the url list:
url_lst = list()
counter = 0
for i in range(30):
    url = 'https://www.tripadvisor.com/Hotels-g32655-oa'+ str(counter) + '-Los_Angeles_California-Hotels.html'
    url_lst.append(url)
    counter += 30

time: 920 µs


In [4]:
# Access the links in url_lst and get urls for hotels:
all_links = list()
for url in url_lst:
    
    all_links = all_links + get_link(url)
    
    #Generate 10s before opening next page:
    time.sleep(10)

time: 5min 47s


In [9]:
print('In total there are',len(all_links),'hotels available in Loa Angeles in TripAdvisor at the time of parsing.')

In total there are 546 hotels available in Loa Angeles in TripAdvisor at the time of parsing.
time: 1.03 ms


In [12]:
# Write links to a txt file:
with open("LA_hotels.txt", "w",encoding='utf-8') as file:
    for link in all_links:
        file.write(link+"\n")

time: 2 ms


In [14]:
# Create a folder to store htmls:
path = os.getcwd()
hotels_dir = os.path.join(path,'LA_hotels')
os.mkdir(hotels_dir)

# Read the file content
hotels_data = open('LA_hotels.txt','r',encoding='utf-8')
hotels_string = hotels_data.read()
#close the file
hotels_data.close()

# Get the links of items from the string stored in files, and strap last element
# that contains nothing.
hotels_links = hotels_string.split('\n')
hotels_links = hotels_links[0:len(hotels_links)-1]

time: 1.96 ms


In [15]:
# Downloads each of the pages (URLs) into the folders "LA_hotels"
for j in range(len(hotels_links)):
    try:
        #Extract the url to a soup object.
        page = get_html(hotels_links[j])
    
        #Transfer the html to BeautifulSoup object:
        soup = BeautifulSoup(page)
        
        #Find the hotel name:
        hotel_name = soup.find('h1',{'id':'HEADING'}).get_text()
        
        #Save content into file with corresponding name
        with open("LA_hotels/%s.htm"%hotel_name, "w",encoding='utf-8') as file:
            file.write(page)
                    
        #Pause 2s between queries
        time.sleep(2)
        
    except: # Error handling
        pass
        print("Pass %dth hotel."%(j+1))

Pass 3th hotel.
Pass 81th hotel.
Pass 123th hotel.
Pass 133th hotel.
Pass 149th hotel.
Pass 179th hotel.
Pass 224th hotel.
Pass 230th hotel.
Pass 238th hotel.
Pass 246th hotel.
Pass 259th hotel.
Pass 262th hotel.
Pass 269th hotel.
Pass 323th hotel.
Pass 354th hotel.
Pass 361th hotel.
Pass 376th hotel.
time: 27min 26s


### (2) develop code that reads the saved file and parses the downloaded page and returns the data points you are looking for.

### The following chunk parses hotel name, vendors and prices, overall score, bubble rating of location, cleanliness, service, and value, hotel description, hotel star, number of room, number of reviews, and url of the hotel page for each hotel. Notice that the url of the hotel page comes from url of the 2nd page of review. 

In [2]:
hotel_name_lst = list()
price_dict_lst = list()
score_lst = list()
address_lst = list()
bubble_rating_dict_lst = list()
descritpion_lst = list()
star_lst = list()
room_number_lst = list()
review_num_lst = list()
parsed_links_lst = list()

for filename in os.listdir('LA_hotels/'):
    if filename != '.DS_Store':
        print("Getting info of %s"%filename)
        
        # Store the hotel name
        hotel_name_lst.append(re.findall(r'(.+?).htm',filename)[0])
        
        # Read content stored in html files
        item_data = open('LA_hotels/%s'%filename,'r',encoding='utf-8')
        content = item_data.read()
        item_data.close() #close the file
        
        #Transfer the html to BeautifulSoup object:
        soup = BeautifulSoup(content)

        # -----------------------------Find the price per night and providors----------------------------
        try:
            premium_offers = soup.find_all('div',{'id':'taplc_resp_hr_atf_meta_component_0'})[0].find_all('div',{'class':'premium_offers_area offers'})
            
            # Providors
            providers_sublst = re.findall(r'data-provider="(.+?)"',str(premium_offers[0]))
            
            # Price per night
            price_sublst = re.findall(r'data-pernight="(.+?)"',str(premium_offers[0]))
            
            # Add provider and corresponding price to a dictionary and save the dictionary to a list called price_dict_lst.
            price_dict = dict()
            for i in range(len(price_sublst)):
                price_dict[providers_sublst[i]] = price_sublst[i]
            price_dict_lst.append(price_dict)

        except:
            providers_sublst = None
            price_sublst = None
            price_dict_lst.append(dict())
            
            
        # -----------------------------------Find the score for the hotel----------------------------------
        try:
            score = soup.find('span',{'class':'hotels-hotel-review-about-with-photos-Reviews__overallRating--vElGA'}).get_text()
            
        except:
            score = None
            
        score_lst.append(score) # Store the score to a list
        

        # -----------------------------------Find the adress for the hotel----------------------------------
        try:
            address = soup.find_all('span',{'class':'public-business-listing-ContactInfo__ui_link--1_7Zp public-business-listing-ContactInfo__level_4--3JgmI'})[0].get_text()
            
        except:
            address = None
            
        address_lst.append(address) # Store the address to a list
        
        
        # ----------------Find the bubble ratings for location, cleanliness, service, and value-------------        
        try:
            all_ratings = soup.find_all('div',{'class':'hotels-hotel-review-about-with-photos-Reviews__subratingRow--2u0CJ'})            
            all_crateria = soup.find_all('div',{'class':'hotels-hotel-review-about-with-photos-Reviews__subratingLabel--H8ZI0'})
            
        except:
            all_ratings = None
            all_crateria = None
            
        #Add rating and corresponding crateria to a dictionary and save the dictionary to a list called bubble_rating_dict_lst.
        bubble_rating_dict = dict()
        for i in range(len(all_crateria)):
            rating_str = re.findall(r'lass="ui_bubble_rating bubble_(..)', str(all_ratings[i]))[0]
            format_str = rating_str[0]+'.'+rating_str[1]
            rating_float = float(format_str)
            bubble_rating_dict[all_crateria[i].get_text()] = rating_float
        bubble_rating_dict_lst.append(bubble_rating_dict)
        
        # -----------------------------------Find the description for the hotel-----------------------------        
        try:
            Description = soup.find_all('div',{'id':'ABOUT_TAB'})[0].find_all('div',{'class':'cPQsENeY'})[0].get_text()            
            
        except:
            Description = None
        descritpion_lst.append(Description)
        
        # --------------------------------------Find the star for the hotel--------------------------------        
        try:
            star_and_type = soup.find_all('div',{'class':'hotels-hr-about-layout-TextItem__textitem--2JToc'})
            star_str = re.findall(r'lass="_2TmwtWEr (.+?) uq1qMUbD', str(star_and_type[0]))[0]
            room_number = None # There are two types of html, this type does not contain information of number of room.
            if star_str == 'f33bWmtw':
                star = 5
            elif star_str == '_30WZSV_9':
                star = 4
            elif star_str == '_3RprXHxE':
                star = 3
            elif star_str == '_2MgVjxWG':
                star = 2
            else:
                star = 1
            
        except:
            try:
                info = soup.find_all('div',{'id':'ABOUT_TAB'})[0].find_all('div',{'class':'hotels-hr-about-layout-TextItem__textitem--2JToc'})
                star = int(info[0].get_text()[0])
            except:
                star = 'Here'
            try:
                room_number = int(info[len(info)-1].get_text())
            except:
                room_number = None  
        star_lst.append(star)
        room_number_lst.append(room_number)
        
        
        # --------------------------------Find the number of reviews for the hotel---------------------------        
        try:
            review_num = int(soup.find_all('span',{'class':'hotels-hotel-review-atf-info-parts-Rating__reviewCount--1sk1X'})[0].get_text().split()[0].replace(',',''))
        except:
            review_num = None
        review_num_lst.append(review_num)

        # ---------------------------------------Find the url for the hotel-----------------------------------        
        try:
            raw_link = soup.find_all('div',{'class':'pageNumbers'})[0].find_all('a',{'class':'pageNum cx_brand_refresh_phase2'})[0]
            parts = re.findall(r'href="(.+?)-or5-(.+?html)"',str(raw_link))
            parsed_link = 'https://www.tripadvisor.com/'+parts[0][0]+'-'+parts[0][1]
        except:
            parsed_link = None
        parsed_links_lst.append(parsed_link)

Getting info of 1 Hotel West Hollywood.htm
Getting info of 24 Hour motel.htm
Getting info of 4 Star Motel.htm
Getting info of 777 Motor Inn.htm
Getting info of 818 Hotel + Pool, an Ascend Hotel Collection Member.htm
Getting info of 8th and Ardmore.htm
Getting info of Ace Hotel Downtown Los Angeles.htm
Getting info of Air Venice On the Beach.htm
Getting info of Airtel Plaza Hotel.htm
Getting info of All Star Inn.htm
Getting info of Alta Cienega Motel.htm
Getting info of American Hotel.htm
Getting info of Americas Best Value Inn.htm
Getting info of Anchor Pointe Inn.htm
Getting info of Anderson Estates.htm
Getting info of Antonio Hotel.htm
Getting info of Avalon Hotel Beverly Hills.htm
Getting info of Avenue Hotel an Ascend Hotel Collection.htm
Getting info of Avocado House.htm
Getting info of Banana Bungalow Hollywood.htm
Getting info of Barton Bungalow.htm
Getting info of Baxter 5 Apartments.htm
Getting info of Bayside Hotel.htm
Getting info of Best Inn.htm
Getting info of Best Western

Getting info of Hometown Inn.htm
Getting info of Homewood Suites by Hilton Los Angeles International Airport.htm
Getting info of Horizon Inn Motel.htm
Getting info of Hostelling International.htm
Getting info of Hotel 850 SVB.htm
Getting info of Hotel Angeleno.htm
Getting info of Hotel Aventura.htm
Getting info of Hotel Bel-Air.htm
Getting info of Hotel Beverly Terrace.htm
Getting info of Hotel Carmel.htm
Getting info of Hotel Casa del Mar.htm
Getting info of Hotel Del Flores.htm
Getting info of Hotel Erwin.htm
Getting info of Hotel Figueroa.htm
Getting info of Hotel Hwood.htm
Getting info of Hotel Indigo Los Angeles Downtown.htm
Getting info of Hotel K Town.htm
Getting info of Hotel Normandie.htm
Getting info of Hotel Shangri-La Santa Monica.htm
Getting info of Hotel Silver Lake Los Angeles.htm
Getting info of Howard Johnson Hotel & Suites by Wyndham Reseda.htm
Getting info of Huntley Santa Monica Beach.htm
Getting info of Hyatt Regency Los Angeles International Airport.htm
Getting in

Getting info of Stallion Motel.htm
Getting info of Starlight Inn La Brea.htm
Getting info of Starlight Inn Valley Boulevard.htm
Getting info of Starlight Inn Van Nuys.htm
Getting info of Starlite Cottage Motel.htm
Getting info of Stay.htm
Getting info of Staybridge Suites Chatsworth.htm
Getting info of Stillwell Hotel.htm
Getting info of Stuart Hotel.htm
Getting info of Studio City Court Yard Hotel.htm
Getting info of Studio City Inn.htm
Getting info of Su Casa Venice Beach.htm
Getting info of Sunrise Hotel San Pedro.htm
Getting info of Sunset Gardes Apartment.htm
Getting info of Super 8 by Wyndham Canoga Park.htm
Getting info of Super 8 by Wyndham Los Angeles Airport.htm
Getting info of Super 8 by Wyndham Los Angeles Downtown.htm
Getting info of Super 8 by Wyndham North Hollywood.htm
Getting info of SureStay Collection by Best Western Sunset West Hotel.htm
Getting info of SureStay Hotel by Best Western Santa Monica.htm
Getting info of Sweet Dreams B&B.htm
Getting info of Tarzana Inn.h

### Then we parse first 10 reviews of each hotel. For those have less than 10 reviews, we just parse 5 reviews which are displayed in the first page of that hotel.

#### One thing need to mention is that since one page in TripAdvisior just shows five reviews, so it is very time consuming to parse reviews. Originally we plan to parse 100 reviews for each hotel, however, it will take very long time so we just decided to parse 10 to showcase the ability to parse the reviews.

In [6]:
# For each hotel, we plan to fetch 10 reviews, each page contains 5 reviews, so we have to access 2 urls for one hotel

# Here we define a function that can give the review urls from a given hotel link
def get_review_urls(link, review_num):
    urls_lst = list()
    if review_num >=10:
        for page in range(0,10,5):
            url = re.sub(r'(https://www.tripadvisor.com//.+?-Reviews)-(.+?html)','\\1-or%d-\\2'%page, link)
            urls_lst.append(url)
    elif review_num >=5:
        for page in range(0,review_num-5,5):
            url = re.sub(r'(https://www.tripadvisor.com//.+?-Reviews)-(.+?html)','\\1-or%d-\\2'%page, link)
            urls_lst.append(url)
    return urls_lst

# Then we define a function that can give the reviews from the review urls
def get_reviews(link, review_num):
    review_urls = get_review_urls(link,review_num)
    reviews = list()
    for url in review_urls:
        try:
            soup = BeautifulSoup(get_html(url))
            raw_reviews = soup.find_all('q',{'class':'location-review-review-list-parts-ExpandableReview__reviewText--gOmRC'})
            for raw_review in raw_reviews:
                review = raw_review.get_text()
                reviews.append(review)
                
            #Pause 2s between queries
            time.sleep(2)
        
        except: # Error handling
            pass
            print("Pass %s."%url)
            
    return reviews

time: 2.02 ms


In [18]:
all_reviews = list()
for i in range(len(parsed_links_lst)):
    if parsed_links_lst[i] == None:
        reviews = list()
    else:
        reviews = get_reviews(parsed_links_lst[i], review_num_lst[i])
    all_reviews.append(reviews)

time: 35min 17s


### We will dump dataframe to tables in mysql database directly. So we create dataframe for each table first.

In [4]:
# Create dataframe for "Hotel" table.
Hotel = pd.DataFrame()

# Transform bubble scores from dictionary to lists
location_lst = list()
cleanliness_lst = list()
service_lst = list()
value_lst = list()

for element in bubble_rating_dict_lst:
    try:
        location_lst.append(element['Location'])
    except:
        location_lst.append(None)
    try:
        cleanliness_lst.append(element['Cleanliness'])
    except:
        cleanliness_lst.append(None)
    try:
        service_lst.append(element['Service'])
    except:
        service_lst.append(None)
    try:
        value_lst.append(element['Value'])
    except:
        value_lst.append(None)
        
# Create index for Hotel
HotelId = list()
for i in range(len(hotel_name_lst)):
    HotelId.append((i+1))

# Insert data into the dataframe
Hotel["HotelId"] = HotelId
Hotel["HotelName"] = hotel_name_lst
Hotel["Score"] = score_lst
Hotel["Address"] = address_lst
Hotel["Location"] = location_lst
Hotel["Cleanliness"] = cleanliness_lst
Hotel["Service"] = service_lst
Hotel["Value"] = value_lst
Hotel["Description"] = descritpion_lst
Hotel["Star"] = star_lst
Hotel["NumberOfRoom"] = room_number_lst
Hotel["NumberOfReview"] = review_num_lst

time: 9 ms


In [5]:
# Create dataframe for "Vendor" and "Price" table.
Price = pd.DataFrame()
Vendor = pd.DataFrame()

HotelId_price = list()
Vendors = list()
Prices = list()
for i in range(len(price_dict_lst)):
    sub_dict = price_dict_lst[i]
    sub_vendors = list(sub_dict.keys())
    Vendors = Vendors+sub_vendors
    sub_prices = list(sub_dict.values())
    Prices = Prices+sub_prices
    HotelId_price = HotelId_price + ([i+1]*len(sub_vendors))

Vendor_set = list(set(Vendors))

# Create index for Vendor
VendorId = list()
for i in range(len(Vendor_set)):
    VendorId.append((i+1))
Vendor["VendorId"] = VendorId
Vendor["VendorName"] = Vendor_set

# Inser data into Price dataframe. 
Price["HotelId"] = HotelId_price
Price["VendorName"] = Vendors
Price["Price"] = Prices
Price = Price.merge(Vendor, left_on='VendorName', right_on='VendorName')
Price = Price[["HotelId","VendorId", "Price"]]

time: 18 ms


In [6]:
# Create dataframe for "Review" table.
Review = pd.DataFrame()

HotelId_review = list()
Reviews = list()
for i in range(len(all_reviews)):
    sub_lst = all_reviews[i]
    Reviews = Reviews + sub_lst
    HotelId_review = HotelId_review + ([i+1]*len(sub_lst))

# Create index for Review
ReviewId = list()
for i in range(len(Reviews)):
    ReviewId.append((i+1))
Review["ReviewId"] = ReviewId

# Insert data into Review dataframe
Review["HoetlId"] = HotelId_review
Review["Review"] = Reviews

time: 10 ms


# Part 2: Database

### a) Using Python: Create a SQL database named “TripAdvisor”, In this database, create SQL tables named “hotel”, "vendor", "price", and "review" that can hold data parsed above. 

In [4]:
# Connect to the database.
conn = pymysql.connect(host='localhost',user='root')

# Create a cursor.
cursor = conn.cursor()

# Create 'amazon_SQL' database.
cursor.execute("DROP DATABASE IF EXISTS TripAdvisor")
cursor.execute("CREATE DATABASE TripAdvisor")

# Check whether the database has be successfully created.
cursor.execute("SHOW DATABASES")

for x in cursor:
    print(x)

('information_schema',)
('mysql',)
('performance_schema',)
('sys',)
('tripadvisor',)
time: 55 ms


In [5]:
sqlEngine = create_engine('mysql+pymysql://root:@127.0.0.1/tripadvisor', pool_recycle=3600)
dbConnection = sqlEngine.connect()
tableName = "Hotel"
Hotel.to_sql(tableName, dbConnection, if_exists='replace', index = False)
tableName = "Vendor"
Vendor.to_sql(tableName, dbConnection, if_exists='replace', index = False)
tableName = "Price"
Price.to_sql(tableName, dbConnection, if_exists='replace', index = False)
tableName = "Review"
Review.to_sql(tableName, dbConnection, if_exists='replace', index = False)

print("Successful created the tables.")

dbConnection.close()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\io\sql.py:1336: UserWarning: The provided table name 'Hotel' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  warnings.warn(msg, UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\io\sql.py:1336: UserWarning: The provided table name 'Vendor' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  warnings.warn(msg, UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\io\sql.py:1336: UserWarning: The provided table name 'Price' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  warnings.warn(msg, UserWarning)


Successful created the tables.
time: 530 ms


C:\ProgramData\Anaconda3\lib\site-packages\pandas\io\sql.py:1336: UserWarning: The provided table name 'Review' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  warnings.warn(msg, UserWarning)
